## Did Louisville recover from recession more quickly than similar cities?

First we'll import libraries. The altair package can be installed this way: conda install altair --channel conda-forge

In [1]:
import sqlite3
import pandas as pd
import altair as alt
alt.renderers.enable('notebook')

ModuleNotFoundError: No module named 'altair'

Reading in an excel sheet of jobs related data for Louisville and for peer cities (source: Greater Louisville Project). You can use the Download Data button on this website to download a copy: http://greaterlouisvilleproject.org/deep-drivers-of-change/21st-century-jobs/

In [ ]:
jobs_df = pd.read_excel('GLP-Codebook.xlsx', 'Jobs County', index_col=None, na_values=['NA'])

Creating a sql database that can be queried from in the next step.

In [ ]:
jobs_df.to_sql("jobs_table", sqlite3.connect("jobs.db"), if_exists = "replace")


### Mostly pandas, some SQL

Querying back just the unemployment, median earnings, and personal income per capita data by year and city. Only for cities that are currently peers

In [ ]:
con = sqlite3.connect("jobs.db")
jobs_df = pd.read_sql_query("SELECT year, city, unemployment, median_earnings, personal_income_per_cap FROM jobs_table WHERE current = 1", con)
jobs_df.head()

Making year into a datetime, subsetting to just Louisville, peers without Louisville, and finding mean of non-Louisville cities. Note: one nice thing about the groupby and mean code is that it doesn't change for multiple variables or require a loop. I changed the query above to include more than just 'unemployment' as shown in class, but didn't have to change the groupby.

In [ ]:
jobs_df['year'] = pd.to_datetime(jobs_df['year'], format = "%Y") #year column as datetime instead of character
lou_df = jobs_df[(jobs_df.city == "Louisville")] # splitting the dataframe
peer_df = jobs_df[(jobs_df.city != "Louisville")]
mean_df = peer_df.groupby('year', as_index = False).mean() #groupby and aggregate
mean_df['city'] = "Peers"
df = mean_df.append(lou_df)
df


This next code chunk loops over the three variables and graphs them. Jupyter notebook limits visuals to one chart per cell (if someone knows how to override this let me know), so I've saved the charts as separate .html files. They're saved in the working directory.



In [ ]:
var_list = ['unemployment', 'median_earnings', 'personal_income_per_cap']
for var in var_list:
    chart = alt.Chart(df).mark_line().encode(
    x='year',
    y= var,
    color='city') #group data and lines by this variable
    chart.save(var+'_chart.html') #saves to working directory as 'unemployment_chart.html', etc. for each variable


What if we want the charts in the notebook though? One workaround is to write a function that we can then call in separate cells. This function is specific to our current dataframe - the only thing that changes is the name of the column. It looks a lot like the for loop, but instead of automatically looping through a list of variables, we have to pass the variable names to the function. One advantange though is in the possible extenions we could make in the future (e.g. a title for the graph or different axis labels could be an argument that gets passed to a function and so can vary graph by graph).

In [ ]:
def make_graph (var):
    chart = alt.Chart(df).mark_line().encode(
        x='year',
        y= var,
        color='city'
    )
    return chart

In [ ]:
#### Unemployment

In [ ]:
make_graph('unemployment')

In [ ]:
make_graph('median_earnings')

In [ ]:
make_graph('personal_income_per_cap')

#### Mostly SQL, some pandas

Now let's try a more SQL based approach. We'll just worry about unemployment for now.

First, pull down Louisville data and rename

In [ ]:
lou_df = pd.read_sql_query("""
SELECT year, unemployment as 'lou_unemployment'
FROM jobs_table 
WHERE current = 1 AND city = 'Louisville'
""", con)
lou_df

Then pull down peer data, rename, and groupby.

In [ ]:
peer_df = pd.read_sql_query("""
SELECT year, avg(unemployment) as 'peer_unemployment'
FROM jobs_table 
WHERE current = 1 AND city != 'Louisville'
GROUP BY year
""", con)
peer_df

Depending on how complex the work is you can save these new tables back into the same database for future queries. Probably not necessary in this case

In [ ]:
peer_df.to_sql("peer_unemp_table", sqlite3.connect("jobs.db"), if_exists = "replace")
lou_df.to_sql("lou_unemp_table", sqlite3.connect("jobs.db"), if_exists = "replace")

Then we can join the two tables in SQL. Note that when dealing with multiple tables we can specify the field we're trying to pull with table_name.field_name (e.g. lou_unemp_table.year).

In [ ]:
df = pd.read_sql_query("""
SELECT lou_unemp_table.year, lou_unemployment, peer_unemployment
FROM lou_unemp_table
JOIN peer_unemp_table
ON lou_unemp_table.year = peer_unemp_table.year
""", con)
df

Just for reference because the pandas code above appends rows to the dataframe, it is also possible to execute SQL-style joins in pandas

In [ ]:
df = pd.merge(lou_df, peer_df, how = 'outer', left_on = ['year'], right_on = ['year'])
df.head()